In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
import torch.nn as nn
import torch.nn.functional as F
from torch import tensor, device, cuda, optim, concat, FloatTensor, reshape
from torch.utils.data import DataLoader

In [2]:
## load the dataframe and view the first 5 rows
df = pd.read_csv('data/chess_games.csv')
df.head(1)

,game_id,rated,turns,victory_status,winner,time_increment,white_id,white_rating,black_id,black_rating,moves,opening_code,opening_moves,opening_fullname,opening_shortname,opening_response,opening_variation
0,1,False,13,Out of Time,White,15+2,bourgris,1500,a-00,1191,d4 d5 c4 c6 cxd5 e6 dxe6 fxe6 Nf3 Bb4+ Nc3 Ba5...,D10,5,Slav Defense: Exchange Variation,Slav Defense,NaN,Exchange Variation


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20058 entries, 0 to 20057
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   game_id            20058 non-null  int64 
 1   rated              20058 non-null  bool  
 2   turns              20058 non-null  int64 
 3   victory_status     20058 non-null  object
 4   winner             20058 non-null  object
 5   time_increment     20058 non-null  object
 6   white_id           20058 non-null  object
 7   white_rating       20058 non-null  int64 
 8   black_id           20058 non-null  object
 9   black_rating       20058 non-null  int64 
 10  moves              20058 non-null  object
 11  opening_code       20058 non-null  object
 12  opening_moves      20058 non-null  int64 
 13  opening_fullname   20058 non-null  object
 14  opening_shortname  20058 non-null  object
 15  opening_response   1207 non-null   object
 16  opening_variation  14398 non-null  objec

In [4]:
df.drop(['moves', 'opening_code', 'opening_fullname', 'opening_response', 'opening_variation', 'time_increment'], axis=1, inplace=True)

In [5]:
df.head()

,game_id,rated,turns,victory_status,winner,white_id,white_rating,black_id,black_rating,opening_moves,opening_shortname
0,1,False,13,Out of Time,White,bourgris,1500,a-00,1191,5,Slav Defense
1,2,True,16,Resign,Black,a-00,1322,skinnerua,1261,4,Nimzowitsch Defense
2,3,True,61,Mate,White,ischia,1496,a-00,1500,3,King's Pawn Game
3,4,True,61,Mate,White,daniamurashov,1439,adivanov2009,1454,3,Queen's Pawn Game
4,5,True,95,Mate,White,nik221107,1523,adivanov2009,1469,5,Philidor Defense


In [6]:
## encode the categorical features
rated_enc = LabelEncoder()
victory_status_enc = LabelEncoder()
winner_enc = LabelEncoder()
opening_shortname_enc = LabelEncoder()

In [7]:
df_enc = df.copy()

In [8]:
df_enc['rated'] = rated_enc.fit_transform(df['rated'])
df_enc['victory_status'] = victory_status_enc.fit_transform(df['victory_status'])
df_enc['winner'] = winner_enc.fit_transform(df['winner'])
df_enc['opening_shortname'] = opening_shortname_enc.fit_transform(df['opening_shortname'])

In [9]:
df_enc.head()

,game_id,rated,turns,victory_status,winner,white_id,white_rating,black_id,black_rating,opening_moves,opening_shortname
0,1,0,13,2,2,bourgris,1500,a-00,1191,5,110
1,2,1,16,3,0,a-00,1322,skinnerua,1261,4,74
2,3,1,61,1,2,ischia,1496,a-00,1500,3,61
3,4,1,61,1,2,daniamurashov,1439,adivanov2009,1454,3,94
4,5,1,95,1,2,nik221107,1523,adivanov2009,1469,5,83


In [10]:
df_enc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20058 entries, 0 to 20057
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   game_id            20058 non-null  int64 
 1   rated              20058 non-null  int64 
 2   turns              20058 non-null  int64 
 3   victory_status     20058 non-null  int32 
 4   winner             20058 non-null  int32 
 5   white_id           20058 non-null  object
 6   white_rating       20058 non-null  int64 
 7   black_id           20058 non-null  object
 8   black_rating       20058 non-null  int64 
 9   opening_moves      20058 non-null  int64 
 10  opening_shortname  20058 non-null  int32 
dtypes: int32(3), int64(6), object(2)
memory usage: 1.5+ MB


In [11]:
winner_enc.classes_


array(['Black', 'Draw', 'White'], dtype=object)

## Build the MLP model

In [12]:
d = device('cuda' if cuda.is_available() else 'cpu')

In [13]:
## Select the features and labels
enc = OneHotEncoder()
X = df_enc[['rated', 'turns', 'victory_status', 'white_rating', 'black_rating', 'opening_moves', 'opening_shortname']].to_numpy()
Y = enc.fit_transform(df_enc['winner'].to_numpy().reshape(-1,1)).toarray()

In [14]:
##Verify the matrices shapes
print("X dimension: ", X.shape)
print("Y dimension: ", Y.shape)

X dimension:  (20058, 7)
Y dimension:  (20058, 3)


In [15]:
## Build the architecture
class MLP(nn.Module):
    def __init__(self, in_features, out_class):
        super().__init__()
        self.fc1 = nn.Linear(in_features, 7)
        self.fc2 = nn.Linear(7,4)
        self.out = nn.Linear(4,out_class)
        self.dropout = nn.Dropout(.2)
    
    def forward(self,x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = F.softmax(self.out(x), dim=0)

        return x
        

In [16]:
mlp = MLP(7,3).to(d)
mlp

MLP(
  (fc1): Linear(in_features=7, out_features=7, bias=True)
  (fc2): Linear(in_features=7, out_features=4, bias=True)
  (out): Linear(in_features=4, out_features=3, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)

In [17]:
def train(model, train_loader, criterion, optimizer):
    model.train()
    losses = []
    for ds in train_loader:  # Iterate in batches over the training dataset.
        ds = ds.type(FloatTensor)
        ds = ds.to(d)
        out = model(ds[:,:-3])  # Perform a single forward pass.
        loss = criterion(out, ds[:,-3:]) # Compute the loss.
        loss.backward()  # Derive gradients.
        losses.append(loss.item())
        optimizer.step()  # Update parameters based on gradients.
        optimizer.zero_grad()  # Clear gradients.

    return np.mean(losses)

def test(model, loader):
    model.eval()

    y_pred = []
    y_true = []
    for ds in loader:  # Iterate in batches over the training/test dataset.
        ds = ds.type(FloatTensor)
        ds = ds.to(d)
        out = model(ds[:,:-3])  
        
        pred = out.argmax(dim=1)  # Use the class with highest probability.

        y_pred.extend(pred.cpu().numpy().tolist())
        y_true.extend(ds[:,-3:].argmax(dim=1).cpu().numpy().tolist())
        
    print(classification_report(y_true, y_pred, target_names=winner_enc.classes_))

In [19]:

##Train with cross validation (3-folds)
kf = KFold(n_splits=3, random_state=46, shuffle=True)
for i, (train_index, test_index) in enumerate(kf.split(X)):
    print(f"Fold {i}:")

    print(f"  Train: ration={round(len(train_index)/X.shape[0], 2)}")
    print(f"  Test:  ration={round(len(test_index)/X.shape[0], 2)}")

    sc = StandardScaler()

    train_d = concat((tensor(sc.fit_transform(X[train_index.tolist()])), tensor(Y[train_index.tolist()])), dim=1)
    test_d = concat((tensor(sc.transform(X[test_index.tolist()])), tensor(Y[test_index.tolist()])), dim=1)

    ##Load the data fro training and testing
    train_data = DataLoader(train_d, batch_size=32, shuffle=True)
    test_data = DataLoader(test_d, batch_size=32, shuffle=True)

    ##Init the model
    mlp = MLP(7,3).to(d)
    optimizer = optim.Adam(mlp.parameters(), lr=1e-2)
    criterion = nn.CrossEntropyLoss()

    ##Train the model
    for epoch in range(1, 11):
        mean_loss = train(mlp, train_data, criterion, optimizer)
        print("loss: ", mean_loss)

    ##Evaluate on training
    print("Training Evaluation:")
    print("Classifiaction report:")
    test(mlp, train_data)


    ##Evaluate on testing

    print("Testing Evaluation:")
    print("Classifiaction report")
    y_pred = test(mlp, test_data)
    
    print('#'*10)

Fold 0:
  Train: ration=0.67
  Test:  ration=0.33
loss:  1.069837975730166
loss:  1.061638924493744
loss:  1.0609555917493465
loss:  1.0616350684439737
loss:  1.060556614513032
loss:  1.0601946166827918
loss:  1.0606238588191677
loss:  1.0605549786649822
loss:  1.0606460468620775
loss:  1.0598249580871546
Training Evaluation:
Classifiaction report:
              precision    recall  f1-score   support

       Black       0.58      0.66      0.62      6089
        Draw       0.23      0.96      0.37       641
       White       0.73      0.40      0.52      6642

    accuracy                           0.55     13372
   macro avg       0.51      0.67      0.50     13372
weighted avg       0.63      0.55      0.55     13372

Testing Evaluation:
Classifiaction report
              precision    recall  f1-score   support

       Black       0.59      0.69      0.63      3018
        Draw       0.23      0.95      0.37       309
       White       0.74      0.41      0.53      3359

    accu